# `pyLDAvis.sklearn`

pyLDAvis now also supports LDA application from scikit-learn. Let's take a look into this in more detail. We will be using 20 newsgroups dataset as provided by scikit-learn.

In [3]:
from __future__ import print_function

In [5]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF


## Load 20 newsgroups dataset

First, the 20 newsgroups dataset available in sklearn are loaded. The headers, footers and quotes are removed, as always.

In [11]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

def strip_links(x):
    stripped = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", x)
    return stripped

def remove_stopwords(x):
    word_tokens = word_tokenize(x)
    filtered = [w for w in word_tokens if w.lower() not in stop_words if w.isalpha()]
    return " ".join(filtered)

df = pd.read_json('../data/all.json')
df["likes_normalized"] = df["likes"] / df["likes"].max()
df["replies_normalized"] = df["replies"] / df["replies"].max()
df["retweets_normalized"] = df["retweets"] / df["retweets"].max()
df = df[(df["likes_normalized"] > 0.05) | (df["replies_normalized"] > 0.05) | (df["retweets_normalized"] > 0.05)]
# df = df[(df["likes_normalized"] < 0.0001) | (df["replies_normalized"] < 0.0001) | (df["retweets_normalized"] < 0.0001)]
df['text'] = df['text'].apply(lambda x: x.replace('http', ' http'))
df['text'] = df['text'].apply(strip_links)
df['text'] = df['text'].apply(remove_stopwords)
print(len(df))

3898


In [12]:
docs_raw = df['text'].tolist()


## Convert to document-term matrix

Next, the raw documents are converted into document-term matrix, possibly as raw counts or TF-IDF form.

In [13]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
#                                 ngram_range=[1,3],
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(3898, 708)


In [14]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

(3898, 708)


/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


## Fit Latent Dirichlet Allocation models

Finally, the LDA models are fitted.

In [37]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=8, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
# lda_tfidf = LatentDirichletAllocation(n_topics=10, random_state=0)
# lda_tfidf.fit(dtm_tfidf)
lda_tfidf = NMF(n_components=8, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(dtm_tfidf)

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


## Visualzing the models with pyLDAvis

In [42]:
data = pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)
pyLDAvis.save_html(data, '../index.html')
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/sklearn.py:26: RuntimeWarning: invalid value encountered in true_divide
  return dists / dists.sum(axis=1)[:, None]
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift
/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambd

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      43.509219        1       1  0.044198 -0.046067
6      17.125670        1       2 -0.190169 -0.208142
7       8.748491        1       3  0.099653 -0.023603
4       8.686793        1       4 -0.362318  0.107764
5       6.998372        1       5  0.120617 -0.137002
1       6.068033        1       6  0.135876 -0.032263
2       5.231424        1       7  0.078819 -0.021896
3       3.631998        1       8  0.073324  0.361208, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
219   Default  230.000000          going  230.000000  30.0000  30.0000
335   Default  192.000000           like  192.000000  29.0000  29.0000
311   Default  179.000000           know  179.000000  28.0000  28.0000
636   Default  643.000000          trump  643.000000  27.0000  27.0000
465   Default  136.000000      president  136.000000  26.0000  26.0000
524   Default  136.000000        russian  136.000000  25.0000  25.0000
394   Default  183.000000        mueller  183.000000  24.0000  24.0000
681   Default  106.000000          white  106.000000  23.0000  23.0000
269   Default   88.000000          house   88.000000  22.0000  22.0000
387   Default   77.000000          money   77.000000  21.0000  21.0000
567   Default  103.000000         source  103.000000  20.0000  20.0000
54    Default   56.000000      billboard   56.000000  19.0000  19.0000
318   Default   51.000000     laundering   51.000000  18.0000  18.0000
291   Default   56.000000  investigation   56.000000  17.0000  17.0000
512   Default   51.000000          right   51.000000  16.0000  16.0000
123   Default   34.000000          crime   34.000000  15.0000  15.0000
664   Default   44.000000           want   44.000000  14.0000  14.0000
361   Default   40.000000       manafort   40.000000  13.0000  13.0000
346   Default   25.000000          looks   25.000000  12.0000  12.0000
532   Default   49.000000           says   49.000000  11.0000  11.0000
98    Default   60.000000          comey   60.000000  10.0000  10.0000
401   Default   73.000000           need   73.000000   9.0000   9.0000
430   Default   27.000000      organized   27.000000   8.0000   8.0000
191   Default   43.000000          flynn   43.000000   7.0000   7.0000
312   Default   29.000000      knowledge   29.000000   6.0000   6.0000
357   Default   37.000000           make   37.000000   5.0000   5.0000
441   Default   56.000000         people   56.000000   4.0000   4.0000
480   Default  100.000000          putin  100.000000   3.0000   3.0000
67    Default   16.000000       business   16.000000   2.0000   2.0000
492   Default   33.000000         really   33.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
155    Topic8    2.816620        elected    5.671102   2.6155  -4.5327
411    Topic8    0.927849        nuclear    2.101609   2.4978  -5.6432
18     Topic8    1.278388      amendment    2.899186   2.4966  -5.3227
440    Topic8    3.904827          pence    8.969846   2.4837  -4.2061
419    Topic8    0.869405       offering    2.189521   2.3918  -5.7082
556    Topic8    0.869772          short    2.196230   2.3891  -5.7078
60     Topic8    1.849923          brain    4.789843   2.3640  -4.9531
65     Topic8    1.341110        brought    3.478883   2.3622  -5.2748
90     Topic8    0.536734        clearly    1.418820   2.3433  -6.1905
617    Topic8    1.186952         threat    3.229905   2.3143  -5.3969
398    Topic8    0.706967       national    2.413197   2.0877  -5.9151
351    Topic8    2.202191           lots    7.705806   2.0629  -4.7788
405    Topic8    1.292714           nice    4.563334   2.0541  -5.3115
489    Topic8    4.182517           real   16.534752   1.9408  -4.1374
693    Topic8    1.357229         worked    5.589594   1.8999  -5.2628
263    To

### Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [39]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      15.190526        1       1 -0.155526  0.332808
2      13.927228        1       2 -0.350592  0.105480
7      13.690891        1       3  0.144974  0.024042
3      13.491932        1       4  0.399486  0.017902
0      11.762675        1       5 -0.238365 -0.122662
6      11.171489        1       6  0.222175 -0.354010
1      11.123583        1       7  0.154404  0.358438
4       9.641676        1       8 -0.176556 -0.361999, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
524   Default   186.000000        russian   186.000000  30.0000  30.0000
401   Default   133.000000           need   133.000000  29.0000  29.0000
98    Default   120.000000          comey   120.000000  28.0000  28.0000
269   Default   118.000000          house   118.000000  27.0000  27.0000
530   Default    97.000000            say    97.000000  26.0000  26.0000
681   Default   119.000000          white   119.000000  25.0000  25.0000
567   Default   122.000000         source   122.000000  24.0000  24.0000
311   Default   144.000000           know   144.000000  23.0000  23.0000
465   Default   154.000000      president   154.000000  22.0000  22.0000
387   Default   102.000000          money   102.000000  21.0000  21.0000
328   Default   111.000000            let   111.000000  20.0000  20.0000
523   Default    86.000000         russia    86.000000  19.0000  19.0000
579   Default    76.000000          state    76.000000  18.0000  18.0000
512   Default   111.000000          right   111.000000  17.0000  17.0000
222   Default   121.000000           good   121.000000  16.0000  16.0000
219   Default   201.000000          going   201.000000  15.0000  15.0000
480   Default   168.000000          putin   168.000000  14.0000  14.0000
191   Default    77.000000          flynn    77.000000  13.0000  13.0000
404   Default    75.000000           news    75.000000  12.0000  12.0000
357   Default    82.000000           make    82.000000  11.0000  11.0000
223   Default    77.000000            gop    77.000000  10.0000  10.0000
664   Default    91.000000           want    91.000000   9.0000   9.0000
636   Default  1339.000000          trump  1339.000000   8.0000   8.0000
54    Default   102.000000      billboard   102.000000   7.0000   7.0000
610   Default    60.000000         thanks    60.000000   6.0000   6.0000
441   Default   123.000000         people   123.000000   5.0000   5.0000
291   Default    70.000000  investigation    70.000000   4.0000   4.0000
673   Default    64.000000            way    64.000000   3.0000   3.0000
361   Default    70.000000       manafort    70.000000   2.0000   2.0000
648   Default    58.000000        twitter    58.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
644    Topic8    34.839485          tweet    35.685408   2.3151  -3.9738
73     Topic8    33.442750       campaign    34.288913   2.3141  -4.0147
596    Topic8    32.534924         taking    33.380586   2.3134  -4.0423
536    Topic8    28.249295         sealed    29.096586   2.3095  -4.1835
194    Topic8    27.579370         follow    28.425058   2.3089  -4.2075
375    Topic8    27.105561         mental    27.951213   2.3084  -4.2248
382    Topic8    26.944592           mind    27.790480   2.3082  -4.2308
675    Topic8    26.536615        weapons    27.382162   2.3077  -4.2461
232    Topic8    26.389815            gun    27.235354   2.3075  -4.2516
601    Topic8    26.226198            tax    27.072871   2.3073  -4.2578
95     Topic8    26.031362      collusion    26.877411   2.3071  -4.2653
472    Topic8    23.415254        problem    24.260844   2.3036  -4.3712
282    Topic8    22.345528     indictment    23.191210   2.3019  -4.4180
359    Topic8    21.225966         making    22.072368   2.

In [40]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
5      15.190526        1       1 -126.533981  -21.990822
2      13.927228        1       2   49.616280  -68.850616
7      13.690891        1       3  -93.960991  -96.283989
3      13.491932        1       4  -15.567182 -117.136879
0      11.762675        1       5  -33.111591  -25.317944
6      11.171489        1       6   -9.079092   65.022499
1      11.123583        1       7   52.501240   12.218026
4       9.641676        1       8  -88.757355   49.796169, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
524   Default   186.000000        russian   186.000000  30.0000  30.0000
401   Default   133.000000           need   133.000000  29.0000  29.0000
98    Default   120.000000          comey   120.000000  28.0000  28.0000
269   Default   118.000000          house   118.000000  27.0000  27.0000
530   Default    97.000000            say    97.000000  26.0000  26.0000
681   Default   119.000000          white   119.000000  25.0000  25.0000
567   Default   122.000000         source   122.000000  24.0000  24.0000
311   Default   144.000000           know   144.000000  23.0000  23.0000
465   Default   154.000000      president   154.000000  22.0000  22.0000
387   Default   102.000000          money   102.000000  21.0000  21.0000
328   Default   111.000000            let   111.000000  20.0000  20.0000
523   Default    86.000000         russia    86.000000  19.0000  19.0000
579   Default    76.000000          state    76.000000  18.0000  18.0000
512   Default   111.000000          right   111.000000  17.0000  17.0000
222   Default   121.000000           good   121.000000  16.0000  16.0000
219   Default   201.000000          going   201.000000  15.0000  15.0000
480   Default   168.000000          putin   168.000000  14.0000  14.0000
191   Default    77.000000          flynn    77.000000  13.0000  13.0000
404   Default    75.000000           news    75.000000  12.0000  12.0000
357   Default    82.000000           make    82.000000  11.0000  11.0000
223   Default    77.000000            gop    77.000000  10.0000  10.0000
664   Default    91.000000           want    91.000000   9.0000   9.0000
636   Default  1339.000000          trump  1339.000000   8.0000   8.0000
54    Default   102.000000      billboard   102.000000   7.0000   7.0000
610   Default    60.000000         thanks    60.000000   6.0000   6.0000
441   Default   123.000000         people   123.000000   5.0000   5.0000
291   Default    70.000000  investigation    70.000000   4.0000   4.0000
673   Default    64.000000            way    64.000000   3.0000   3.0000
361   Default    70.000000       manafort    70.000000   2.0000   2.0000
648   Default    58.000000        twitter    58.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
644    Topic8    34.839485          tweet    35.685408   2.3151  -3.9738
73     Topic8    33.442750       campaign    34.288913   2.3141  -4.0147
596    Topic8    32.534924         taking    33.380586   2.3134  -4.0423
536    Topic8    28.249295         sealed    29.096586   2.3095  -4.1835
194    Topic8    27.579370         follow    28.425058   2.3089  -4.2075
375    Topic8    27.105561         mental    27.951213   2.3084  -4.2248
382    Topic8    26.944592           mind    27.790480   2.3082  -4.2308
675    Topic8    26.536615        weapons    27.382162   2.3077  -4.2461
232    Topic8    26.389815            gun    27.235354   2.3075  -4.2516
601    Topic8    26.226198            tax    27.072871   2.3073  -4.2578
95     Topic8    26.031362      collusion    26.877411   2.3071  -4.2653
472    Topic8    23.415254        problem    24.260844   2.3036  -4.3712
282    Topic8    22.345528     indictment    23.191210   2.3019  -4.4180
359    Topic8    21